In [ ]:
#%load_ext autoreload
#%reload_ext autoreload
#%autoreload 2

#simple moving average crossover strategy
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import time


#function to send a market order
def market_order(symbol, volume, order_type, **kwargs):
    tick = mt5.symbol_info_tick(symbol)

    order_dict = {'buy': 0, 'sell':1}
    price_dict = {'buy': tick.ask, 'sell': tick.bid}

    #send order to the market
    #documentation: https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume, #FLOAT
        "type": order_dict[order_type],
        "price": price_dict[order_type],
        #"sl": 0.0, #FLOAT
        #"tp": 0.0, #FLOAT
        "deviation": DEVIATION, #INTEGER
        "magic": 100, #INTEGER
        "comment": "python market order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    order_result = mt5.order_send(request)
    print(order_result)

    return order_result

#function to close an order based on ticket id
def close_order(ticket):
    positions = mt5.positions_get()

    for pos in positions:
        tick = mt5.symbol_info_tick(pos.symbol)
        type_dict = {0: 1, 1: 0} # 0 represents buy, 1 represents sell - inverting order_type to close the position
        price_dict = {0: tick.ask, 1:tick.bid}

        if pos.ticket == ticket:
            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "position": pos.ticket,
                "symbol": pos.symbol,
                "volume": pos.volume,
                "type": type_dict[pos.type],
                "price": price_dict[pos.type],
                "deviation": DEVIATION,
                "magic": 100,
                "comment": "python close order",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            order_result = mt5.order_send(request)
            print(order_result)

            return order_result

    return 'Ticket does not exist'


#function to get the exposure of a symbol
def get_exposure(symbol):
    positions = mt5.positions_get(symbol = symbol)
    if positions:
        pos_df = pd.DataFrame(positions, columns = positions[0]._asdict().keys())
        exposure = pos_df['volume'].sum()

        return exposure


#function to look for trading signals
def signal(symbol, timeframe, sma_period):
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 1, sma_period) # 1 represents data of candles from start
    bars_df = pd.DataFrame(bars)
    
    last_close = bars_df.iloc[-1]['close']
    sma = bars_df['close'].mean()

    direction = 'flat'
    if last_close > sma:
        direction = 'buy'
    elif last_close < sma:
        direction = 'sell'

    return last_close, sma, direction

if __name__ == '__main__':

    #strategy parameters
    SYMBOL = "EURUSD"
    VOLUME = 0.01
    TIMEFRAME = mt5.TIMEFRAME_M1
    SMA_PERIOD = 10
    DEVIATION = 20

    mt5.initialize()

    while True: 
        #calculating account expsoure
        exposure = get_exposure(SYMBOL)

        #calculating last candle close and simple moving average and checking for trading signal
        last_close, sma, direction = signal(SYMBOL, TIMEFRAME, SMA_PERIOD)

        #trading logic
        if direction == 'buy':
            #if we have a BUY signal, close all short positions
            for pos in mt5.positions_get():
                if pos.type == 1: #pos.type == 1 represent a sell order
                    close_order(pos.ticket)

            #if there are no open positions, open a new long position
            if not mt5.positions_total():
                market_order(SYMBOL, VOLUME, direction)

        elif direction =='sell':
            #if we have a SELL signal, close all buy positions
            for pos in mt5.positions_get():
                if pos.type ==0: #pos.type == 0 represent a buy order
                    close_order(pos.ticket)
            
            #if there are no open positions, open a new short position
            if not mt5.positions_total():
                market_order(SYMBOL, VOLUME, direction)
        
        print('time:', datetime.now())
        print('exposure:', exposure)
        #print('last close:', last close)
        print('sma:', sma)
        print('signal:', direction)
        print('-----\n')
        
        
        



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
OrderSendResult(retcode=10004, deal=0, order=0, volume=0.0, price=0.0, bid=1.07136, ask=1.07136, comment='Requote', request_id=978158494, retcode_external=0, request=TradeRequest(action=1, magic=100, order=0, symbol='EURUSD', volume=0.01, price=1.07135, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))
time: 2024-06-20 13:52:53.573228
exposure: None
sma: 1.0716810000000003
signal: sell
-----

OrderSendResult(retcode=10009, deal=51310813681, order=51330680483, volume=0.01, price=1.07136, bid=1.07136, ask=1.07136, comment='Request executed', request_id=978158495, retcode_external=0, request=TradeRequest(action=1, magic=100, order=0, symbol='EURUSD', volume=0.01, price=1.07136, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python market orde